# Projective Preferential Bayesian Optimization: DSGE prior elicitation

<font size="4">In this notebook the expert knowledge about DSGE priors can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The test case is the New Keynesian sticky price model of Calvo (1983). </font> 

#### Import dependencies

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
%matplotlib widget

In [3]:
import os
import sys
sys.path.insert(1, os.getcwd()+'/src')
sys.path.insert(1, os.getcwd()+'/dsge')
import time
from datetime import datetime
import numpy as np
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query
from gui import GUI_session
from jupyter_ui_poll import run_ui_poll_loop
from ipywidgets.widgets import HBox
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 45em; }</style>")) #Make outputwindow larger

#### Specify the aquisition strategy and the problem setting
Acquisition startegies with unit projections ($\boldsymbol{\xi}$ is an unit vector):
- PCD = preferential coordinate descent
- EI-EXT = same as EI-FIXEDX except only unit projections are allowed
- EI-EXT-FAST = same as EI-EXT except $d\mathbf{x}$ integral omitted
- EI-VARMAX = same as EI-EXT except $\mathbf{x}$ is chosen to maximize GP variance
- EI-VARMAX-FAST = same as EI-VARMAX except $d\mathbf{x}$ integral omitted

Acquisition startegies with non-unit projections:
- EI = expected improvement by projective preferential query
- EI-FIXEDX = same as EI except $\mathbf{x}$ is fixed to $\textrm{argmax}_{\mathbf{x}}\mu(\mathbf{x})$ (xstar)
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

'user_feedback_grid_size' determines the granularity of the slider, that is, the number of options available for the user. Each option is run in a dedicated CPU. Hence, the optimal performance will be achieved if 'user_feedback_grid_size' is a multiple of the number of CPUs. 

In [ ]:
acquisition_strategy = 'EI-VARMAX-FAST'

In [4]:
PPBOset = PPBO_settings(D=7,bounds=((0.2,0.8),(0.2,0.8),(0.2,0.8),(0.2,0.8),(2,15),(0.25,3),(0.3,0.9)),
                    xi_acquisition_function=acquisition_strategy,user_feedback_grid_size=10,verbose=False)

Unknown acquisition function!


#### Set initial queries

In [5]:
initial_queries_xi = np.diag([PPBOset.original_bounds[i][1] for i in range(PPBOset.D)])
initial_queries_x = np.random.uniform([PPBOset.original_bounds[i][0] for i in range(PPBOset.D)], [PPBOset.original_bounds[i][1] for i in range(PPBOset.D)], (len(initial_queries_xi), PPBOset.D))

#### Querying settings

In [6]:
NUMBER_OF_QUERIES = 12
ADAPTIVE_INITIALIZATION = True  #In initilization: immediatly update the coordinate according to the user feedback

#### Hyperparameter optimization

In [7]:
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER = 1000 

#### Initialize the user session

In [8]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('dsge/user_session_log_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI_session(PPBOset)
results_mustar = []
results_xstar = []
results_xstar_unscaled = []

## Elicitation loop

In [9]:
start = time.time() 

In [10]:
for i in range(len(initial_queries_xi)+NUMBER_OF_QUERIES):
    ''' Next projective preferential query '''
    if i < len(initial_queries_xi):
        xi = initial_queries_xi[i].copy()
        if not i==0 and GUI_ses.user_feedback is not None and ADAPTIVE_INITIALIZATION:
            initial_queries_x[i:,:] = GUI_ses.user_feedback
        x = initial_queries_x[i].copy()
        x[xi!=0] = 0
    else:
        xi,x = next_query(PPBOset,GP_model,unscale=True)
    GUI_ses.set_x(x)
    GUI_ses.set_xi(xi)
    ''' Event loop '''
    button,slider,fig,l1,l2,l3,l4 = GUI_ses.prepare_app()
    app = HBox(children=(slider,button))
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            GUI_ses.update_plot(l1,l2,l3,l4,fig,slider)  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Create GP model for first time '''
    if i==0:
        GP_model = GPModel(PPBOset)
    ''' Update GP model '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.update_data()
    GP_model.update_model()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER:
        GP_model.update_model(optimize_theta=True)     
    else:
        GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    ''' Save the predictive mean maximum and maximizer'''
    results_mustar.append(GP_model.mustar)
    results_xstar.append(GP_model.xstar)
    results_xstar_unscaled.append(GP_model.FP.unscale(GP_model.xstar)) 
print("Elicitation done!")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Invalid acquisition function name!


TypeError: cannot unpack non-iterable int object

In [ ]:
print("---------------------------------------")
print("The session completed!")
print("Total time: " + str(time.time()-start) + " seconds.")
xstar, mu_star_  = GP_model.mu_star(mustar_finding_trials=10)
xstar_unscaled = GP_model.FP.unscale(xstar)
print("The final xstar: " + str(xstar))

## Save the results

#### Save the user session results

In [ ]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results.copy()
user_session_results['iter_mustar'] = results_mustar
user_session_results['iter_xstar'] = results_xstar
user_session_results['iter_xstar_unscaled'] = results_xstar_unscaled
user_session_results.to_csv('dsge/user_session_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

## Analyze the results

#### The user's most preferred configuration

In [ ]:
print(str(xstar_unscaled))

#### Find parametric priors that are in KL-sense closest to the expert's opinion

In [ ]:
from priorelicitation import Prior, g_theta, minimize_KL, sample_h, plot
from random_fourier_sampler import Hsampler
from scipy.stats import norm, beta
from matplotlib import pyplot as plt

In [ ]:
h_sampler = Hsampler(GP_model)
h_sampler.generate_basis()
h_sampler.update_phi_X()
h_sampler.update_omega_MAP()
h_sampler.update_covariancematrix()

In [ ]:
#lambda_sample = sample_h(GP_model,sample_size=100,n_input_points=1000)
lambda_sample=[]
for i in range(1000):
    lambda_sample.append(GP_model.FP.unscale(h_sampler.sample_xstar()))
lambda_sample = np.array(lambda_sample)

In [ ]:
a = np.array(a)
a

In [ ]:
fixed_hyperparams = [0.2,0.2,0.2,0.2,1.5,0.375,0.1] #Scale parameters of the priors
prior0 = Prior(family='beta',valrange=(0,1),lambda_indices=[0],fixed_hyperparams=[fixed_hyperparams[0]],is_location_param=True)
prior1 = Prior(family='beta',valrange=(0,1),lambda_indices=[1],fixed_hyperparams=[fixed_hyperparams[1]],is_location_param=True) 
prior2 = Prior(family='beta',valrange=(0,1),lambda_indices=[2],fixed_hyperparams=[fixed_hyperparams[2]],is_location_param=True) 
prior3 = Prior(family='beta',valrange=(0,1),lambda_indices=[3],fixed_hyperparams=[fixed_hyperparams[3]],is_location_param=True) 
prior4 = Prior(family='normal',valrange=(2,15),lambda_indices=[4],fixed_hyperparams=[fixed_hyperparams[4]],is_location_param=True) 
prior5 = Prior(family='normal',valrange=(0.25,3),lambda_indices=[5],fixed_hyperparams=[fixed_hyperparams[5]],is_location_param=True) 
prior6 = Prior(family='beta',valrange=(0,1),lambda_indices=[6],fixed_hyperparams=[fixed_hyperparams[6]],is_location_param=True) 

Prior0: Beta(x,0.2) for parameter $\rho_{Z}$

In [ ]:
thetas = np.arange(prior0.domain[0], prior0.domain[1], 0.01)
g = [g_theta(theta,prior0,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior0)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior0.fixed_hyperparams[0]),g,thetas,fig_id=10)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.977, prior0.fixed_hyperparams[0]), c='green')

Prior1: Beta(x,0.2) for parameter $\rho_{g}$

In [ ]:
thetas = np.arange(prior1.domain[0], prior1.domain[1], 0.005)
g = [g_theta(theta,prior1,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior1)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior1.fixed_hyperparams[0]),g,thetas,fig_id=11)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.957, prior1.fixed_hyperparams[0]), c='green')

Prior2: Beta(x,0.2) for parameter $\theta_{p}$

In [ ]:
thetas = np.arange(prior2.domain[0], prior2.domain[1], 0.005)
g = [g_theta(theta,prior2,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior2)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior2.fixed_hyperparams[0]),g,thetas,fig_id=12)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.648, prior2.fixed_hyperparams[0]), c='green')

Prior3: Beta(x,0.2) for parameter $\theta_{w}$

In [ ]:
thetas = np.arange(prior3.domain[0], prior3.domain[1], 0.005)
g = [g_theta(theta,prior3,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior3)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior3.fixed_hyperparams[0]),g,thetas,fig_id=13)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.649, prior3.fixed_hyperparams[0]), c='green')

Prior4: N(x,1.5) for parameter inv_adj_cost

In [ ]:
thetas = np.arange(prior4.domain[0], prior4.domain[1], 0.01)
g = [g_theta(theta,prior4,lambda_sample) for theta in thetas]
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior4)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior4.fixed_hyperparams[0]),g,thetas,fig_id=14)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 5.632, prior4.fixed_hyperparams[0]), c='green')

Prior5: N(x,0.375) for parameter $\sigma_c$

In [ ]:
thetas = np.arange(prior5.domain[0], prior5.domain[1], 0.01)
g = [g_theta(theta,prior5,lambda_sample) for theta in thetas]
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior5)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior5.fixed_hyperparams[0]),g,thetas,fig_id=15)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 1.267, prior5.fixed_hyperparams[0]), c='green')

Prior6: Beta(x,0.1) for parameter $h$

In [ ]:
thetas = np.arange(prior6.domain[0], prior6.domain[1], 0.005)
g = [g_theta(theta,prior6,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior6)

In [ ]:
#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior6.fixed_hyperparams[0]),g,thetas,fig_id=16)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.504, prior6.fixed_hyperparams[0]), c='green')